In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import h5py
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import glob
import re
from multiprocessing import cpu_count
from multiprocessing import Pool

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import os
import sys
from functools import partial

from sklearn.metrics import confusion_matrix

In [3]:
# find GPU device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print (torch.cuda.device_count())

1


In [5]:
dataDir = 'dataFiles'
datasetSize = '100K'
earthquakeSampleFraction = 0.25
noiseSampleFraction = 0.25
csvFileName = f'{dataDir}/filtered_earthquakeSampleFraction_{earthquakeSampleFraction}_noiseSampleFraction_{noiseSampleFraction}.csv'
hdf5FileName = f'{dataDir}/filtered_earthquakeSampleFraction_{earthquakeSampleFraction}_noiseSampleFraction_{noiseSampleFraction}.hdf5'

trainSetFraction = 0.8
valSetFraction = 0.1
testSetFraction = 1 - (trainSetFraction + valSetFraction) 

# read the csv file
df_csv = pd.read_csv(csvFileName)

# set trace name as index
df_csv.set_index(['trace_name'], inplace=True)

# keep rows with valid data
df_csv.drop(df_csv[df_csv['p_arrival_sample'].isna()].index, inplace=True)

# split the dataset into train, validation and test
df_train, df_val, df_test = np.split(df_csv.sample(frac = 1), [int(trainSetFraction*len(df_csv)), int((trainSetFraction + valSetFraction)*len(df_csv))])

# read the hdf5 file
hdf5Data = h5py.File(hdf5FileName, 'r')

In [6]:
print ('Train set length =', len(df_train))
print ('Val set length =', len(df_val))
print ('Test set length =', len(df_test))

Train set length = 40000
Val set length = 5000
Test set length = 5000


In [7]:
# define other relevant dirNames

# dir to dump plots
plotImgDir = 'plotImages/cnn1dPArrivalRegression'
os.system(f'mkdir -p {plotImgDir}')

# directories to dump neuralNet params
netParamsDirName = 'netParams/cnn1dPArrivalRegression'
os.system(f'mkdir -p {netParamsDirName}')

0

In [8]:
# data loader

class Dataset(torch.utils.data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, listIDs, labels):
        'Initialization'
        self.labels = labels
        self.listIDs = listIDs
        

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.listIDs)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.listIDs[index]

        # Load data and get label
        X = hdf5Data.get('data/' + ID)
        X = np.array(X)
        X = X.T
        y = self.labels[ID]

        return X, y

In [9]:
# define a 1D convolutional neural network

dropOutProb = 0.5

class Net(nn.Module):
    def __init__(self, dropOutProb = 0.5):
        
        # inherit base class
        super(Net, self).__init__()
        
        # layer 1
        self.conv1 = nn.Conv1d(in_channels = 3, out_channels = 12, kernel_size=5, stride = 1, padding = 2)
                
        # layer 2
        self.conv2 = nn.Conv1d(in_channels = 12, out_channels = 12, kernel_size=5, stride = 1, padding = 2)
                
        # layer 3
        self.conv3 = nn.Conv1d(in_channels = 12, out_channels = 8, kernel_size=5, stride = 1, padding = 2)
        
        # use downsampling by 2 for 1st 2 layers, and then downsample by 4 for the 3rd
        self.poolDiv2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.poolDiv4 = nn.MaxPool1d(kernel_size=4, stride=4)
        
        # dropout layer
        self.dropOut = nn.Dropout(p = dropOutProb)
        
        # fully connected layers
        self.fc1 = nn.Linear(in_features = 375 * 8, out_features = 16)
        self.fc2 = nn.Linear(in_features = 16, out_features = 16)
        self.fc3 = nn.Linear(in_features = 16, out_features = 1)

    def forward(self, x):
        x = self.poolDiv2(F.relu(self.conv1(x)))
        x = self.poolDiv2(F.relu(self.conv2(x)))
        x = self.poolDiv4(F.relu(self.conv3(x)))
        x = x.view(-1, self.num_flat_features(x))
        x = self.dropOut(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):        
        size = x.size()[1:]
        num_features = 1        
        for s in size:
            num_features *= s            
        return num_features


net = Net(dropOutProb)
print(net.parameters)

<bound method Module.parameters of Net(
  (conv1): Conv1d(3, 12, kernel_size=(5,), stride=(1,), padding=(2,))
  (conv2): Conv1d(12, 12, kernel_size=(5,), stride=(1,), padding=(2,))
  (conv3): Conv1d(12, 8, kernel_size=(5,), stride=(1,), padding=(2,))
  (poolDiv2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (poolDiv4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (dropOut): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=3000, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=1, bias=True)
)>


In [10]:
# Loss criteria
criterion = nn.MSELoss()

In [11]:
# convergence params
movingWindowSize = 20

# multiprocessing cpu params
numWorkers = 4

# define dataset variables to adapt to dataloader format

# training set. divide into batches inside the training loop
trainingListIDs = df_train.index.get_level_values('trace_name').tolist()
trainingLabels = (df_train['p_arrival_sample']).to_dict()


# validation set
validationListIDs = df_val.index.get_level_values('trace_name').tolist()
validationLabels = (df_val['p_arrival_sample']).to_dict()

validationSet = Dataset(validationListIDs, validationLabels)
validationGenerator = torch.utils.data.DataLoader(validationSet, batch_size = 256, num_workers = numWorkers)

# test set
testListIDs = df_test.index.get_level_values('trace_name').tolist()
testLabels = (df_test['p_arrival_sample']).to_dict()

testSet = Dataset(testListIDs, testLabels)
testGenerator = torch.utils.data.DataLoader(testSet, batch_size = 256, num_workers = numWorkers)
    
# hyperParameters
numEpochs = 200
batchSize = 64
learningRateList = [1e-1, 1e-2, 1e-3, 1e-4]
dropOutProbList = [0.3, 0.5, 0.7]
#learningRateList = [1e-3]
#dropOutProbList = [0.5]

# training
lossProgression = np.zeros((len(dropOutProbList), len(learningRateList), numEpochs))
lossProgression[:] = np.nan

trainLossProgression = np.zeros((len(dropOutProbList), len(learningRateList), numEpochs))
trainLossProgression[:] = np.nan

valLossProgression = np.zeros((len(dropOutProbList), len(learningRateList), numEpochs))
valLossProgression[:] = np.nan

bestValLoss = np.zeros((len(dropOutProbList), len(learningRateList)))
bestValLoss[:] = np.inf


for d,dropOutProb in enumerate (dropOutProbList):
    
    numBatches = int(len(df_train)/batchSize) + 1 if len(df_train) % batchSize != 0 else int(len(df_train)/batchSize)
    
    # invoke dataloader with appropriate batchSize
    trainingSet = Dataset(trainingListIDs, trainingLabels)
    trainingGenerator = torch.utils.data.DataLoader(trainingSet, batch_size = batchSize, shuffle=True, num_workers = numWorkers)        
    
    for l,learningRate in enumerate(learningRateList):
        
        # create a neuralNet object
        net = Net(dropOutProb)        
        net.to(device)
        
        
        # optimizer type
        optimizer = optim.Adam(net.parameters(), lr=learningRate)
        
        # movingWindow of previous epochs to check for convergence
        deltaValLossHistory = []
        prevValLoss = np.inf        
        
        fileTag = f'batchSize_{batchSize}_learningRate_{learningRate}_dropOutProb_{dropOutProb}_numEpochs_{numEpochs}_dataFraction_{earthquakeSampleFraction}_{noiseSampleFraction}'
        
        for epoch in range(numEpochs):                        
            
            # set network in training mode
            net.train()
            
            runningLoss = 0.0
            sqError = 0.0
            for i, data in enumerate(trainingGenerator, 0):

                inputBatch, groundTruthBatch = data
                groundTruthBatch = torch.tensor(groundTruthBatch, dtype = torch.float32)
                inputBatch, groundTruthBatch = inputBatch.to(device), groundTruthBatch.to(device)
                                                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward pass
                outputPredBatch = net(inputBatch)
                
                
                # compute loss
                loss = criterion(outputPredBatch,groundTruthBatch.unsqueeze(1))
                
                # backprop
                loss.backward()
                
                # gradient descent step
                optimizer.step()    
                
                # output prediction label
                outputPredLabelBatch = torch.round(outputPredBatch)

                # collect training loss, accuracy statistics
                if i == numBatches - 1:
                    runningLoss += loss.item() * (len(df_train) - i*batchSize)                    
                else:                        
                    runningLoss += loss.item() * batchSize
                
                sqError += torch.sum((outputPredLabelBatch.squeeze() - groundTruthBatch)**2)
            
            # track training loss/accuracy Progression
            avgLoss = runningLoss/len(df_train)
            lossProgression[d, l, epoch] = avgLoss
            trainLoss = sqError/len(df_train)
            trainLossProgression[d,l, epoch] = trainLoss
                        
            # evaluate validation set accuracy
            
            # set network in eval mode
            net.eval()
            
            sqError = 0.0
            with torch.no_grad():
                
                for i, data in enumerate(validationGenerator, 0):

                    inputBatch, groundTruthBatch = data
                    groundTruthBatch = torch.tensor(groundTruthBatch, dtype = torch.float32)
                    inputBatch, groundTruthBatch = inputBatch.to(device), groundTruthBatch.to(device)
                    outputPredBatch = net(inputBatch)
                    
                    # output prediction label
                    outputPredLabelBatch = torch.round(outputPredBatch)
                    
                    sqError += torch.sum((outputPredLabelBatch.squeeze() - groundTruthBatch)**2)

            valLoss = sqError/len(df_val)
            valLossProgression[d,l,epoch] = valLoss
                        
            print('[batchSize = %d, dropOutProb = %.2g, learningRate = %.2g, epoch = %d] loss: %f, trainLoss: %f, valLoss: %f' %
                                                (batchSize, dropOutProb, learningRate, epoch + 1, avgLoss, trainLoss, valLoss))
            
            # update network parameters if validation accuracy in current epoch is better than the past            
            if valLoss < bestValLoss[d,l]:
                
                # store the neuralNet parameters in a file                
                path = f'{netParamsDirName}/cnn3Layer_12_12_8_fc2layer_16_16_{fileTag}.pth'
                torch.save(net.state_dict(), path)
                
                bestValLoss[d,l] = valLoss
            
            # check for convergence and exit early if valLoss is not improving
            deltaValLoss = valLoss - prevValLoss
            deltaValLossHistory.insert(0, deltaValLoss)            
            
            if epoch >= movingWindowSize:
                deltaValLossHistory.pop()
                
                if sum(deltaValLossHistory) >= 0:
                    print ('Validation loss starting to increase. Exiting...')
                    break
                
            prevValLoss = valLoss                    
        
        print ('Finished training...')
        print ('\n')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:84: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:131: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


[batchSize = 64, dropOutProb = 0.3, learningRate = 0.1, epoch = 1] loss: 126284399.958950, trainLoss: 126284424.000000, valLoss: 452132.656250
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.1, epoch = 2] loss: 450134.894050, trainLoss: 450131.812500, valLoss: 429140.531250
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.1, epoch = 3] loss: 422510.773300, trainLoss: 422514.906250, valLoss: 397073.750000
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.1, epoch = 4] loss: 387038.729700, trainLoss: 387041.593750, valLoss: 359463.000000
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.1, epoch = 5] loss: 345939.408150, trainLoss: 345942.187500, valLoss: 316393.843750
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.1, epoch = 6] loss: 301901.312000, trainLoss: 301901.625000, valLoss: 272433.312500
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.1, epoch = 7] loss: 257638.348325, trainLoss: 257637.656250, valLoss: 229499.750000
[batchSize = 64, dropOutProb = 0.3,

[batchSize = 64, dropOutProb = 0.3, learningRate = 0.01, epoch = 22] loss: 7659.874566, trainLoss: 7660.108398, valLoss: 7037.210938
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.01, epoch = 23] loss: 7032.658893, trainLoss: 7032.687988, valLoss: 6883.525391
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.01, epoch = 24] loss: 21673.342507, trainLoss: 21673.232422, valLoss: 12956.986328
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.01, epoch = 25] loss: 12896.670115, trainLoss: 12896.823242, valLoss: 11635.053711
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.01, epoch = 26] loss: 11964.704102, trainLoss: 11964.103516, valLoss: 11362.260742
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.01, epoch = 27] loss: 11534.123859, trainLoss: 11533.997070, valLoss: 10840.883789
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.01, epoch = 28] loss: 11083.809438, trainLoss: 11084.042969, valLoss: 10670.361328
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.0

[batchSize = 64, dropOutProb = 0.3, learningRate = 0.001, epoch = 43] loss: 17917.418264, trainLoss: 17917.425781, valLoss: 14652.732422
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.001, epoch = 44] loss: 16329.980146, trainLoss: 16329.479492, valLoss: 13599.817383
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.001, epoch = 45] loss: 17432.578138, trainLoss: 17432.892578, valLoss: 13472.873047
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.001, epoch = 46] loss: 17859.878398, trainLoss: 17859.974609, valLoss: 13826.919922
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.001, epoch = 47] loss: 15623.809535, trainLoss: 15623.754883, valLoss: 11934.632812
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.001, epoch = 48] loss: 17470866.205311, trainLoss: 17470882.000000, valLoss: 18469.621094
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.001, epoch = 49] loss: 19923.665970, trainLoss: 19923.050781, valLoss: 12319.107422
[batchSize = 64, dropOutProb = 0.3,

[batchSize = 64, dropOutProb = 0.3, learningRate = 0.0001, epoch = 34] loss: 77409.838169, trainLoss: 77411.351562, valLoss: 117492.054688
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.0001, epoch = 35] loss: 82068.451375, trainLoss: 82068.085938, valLoss: 76503.265625
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.0001, epoch = 36] loss: 73370.309025, trainLoss: 73370.617188, valLoss: 71226.671875
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.0001, epoch = 37] loss: 66427.938044, trainLoss: 66426.843750, valLoss: 69368.125000
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.0001, epoch = 38] loss: 62343.908844, trainLoss: 62343.320312, valLoss: 67748.117188
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.0001, epoch = 39] loss: 57269.228025, trainLoss: 57269.507812, valLoss: 69070.796875
[batchSize = 64, dropOutProb = 0.3, learningRate = 0.0001, epoch = 40] loss: 56733.296734, trainLoss: 56733.457031, valLoss: 59568.105469
[batchSize = 64, dropOutProb = 0.

...

[batchSize = 64, dropOutProb = 0.5, learningRate = 0.0001, epoch = 20] loss: 99697.699350, trainLoss: 99696.710938, valLoss: 107509.875000
[batchSize = 64, dropOutProb = 0.5, learningRate = 0.0001, epoch = 21] loss: 99025.072069, trainLoss: 99024.054688, valLoss: 101209.421875
[batchSize = 64, dropOutProb = 0.5, learningRate = 0.0001, epoch = 22] loss: 93756.838850, trainLoss: 93757.500000, valLoss: 100562.976562
[batchSize = 64, dropOutProb = 0.5, learningRate = 0.0001, epoch = 23] loss: 88778.518538, trainLoss: 88777.570312, valLoss: 103079.132812
[batchSize = 64, dropOutProb = 0.5, learningRate = 0.0001, epoch = 24] loss: 85980.646738, trainLoss: 85980.015625, valLoss: 88837.796875
[batchSize = 64, dropOutProb = 0.5, learningRate = 0.0001, epoch = 25] loss: 84715.120425, trainLoss: 84712.859375, valLoss: 88032.148438
[batchSize = 64, dropOutProb = 0.5, learningRate = 0.0001, epoch = 26] loss: 104646.723400, trainLoss: 104646.679688, valLoss: 104367.695312
[batchSize = 64, dropOutPro

[batchSize = 64, dropOutProb = 0.5, learningRate = 0.0001, epoch = 80] loss: 32120.429037, trainLoss: 32120.683594, valLoss: 29288.011719
[batchSize = 64, dropOutProb = 0.5, learningRate = 0.0001, epoch = 81] loss: 32594.927917, trainLoss: 32594.818359, valLoss: 36180.421875
[batchSize = 64, dropOutProb = 0.5, learningRate = 0.0001, epoch = 82] loss: 29977.533425, trainLoss: 29977.369141, valLoss: 30622.443359
[batchSize = 64, dropOutProb = 0.5, learningRate = 0.0001, epoch = 83] loss: 28443.048228, trainLoss: 28443.320312, valLoss: 31911.355469
[batchSize = 64, dropOutProb = 0.5, learningRate = 0.0001, epoch = 84] loss: 27039.996158, trainLoss: 27040.482422, valLoss: 33375.488281
[batchSize = 64, dropOutProb = 0.5, learningRate = 0.0001, epoch = 85] loss: 26438.014178, trainLoss: 26437.548828, valLoss: 26897.955078
[batchSize = 64, dropOutProb = 0.5, learningRate = 0.0001, epoch = 86] loss: 27673.562705, trainLoss: 27674.492188, valLoss: 29969.320312
[batchSize = 64, dropOutProb = 0.5

[batchSize = 64, dropOutProb = 0.7, learningRate = 0.01, epoch = 14] loss: 13571.686778, trainLoss: 13571.907227, valLoss: 10915.534180
[batchSize = 64, dropOutProb = 0.7, learningRate = 0.01, epoch = 15] loss: 12922.832682, trainLoss: 12922.818359, valLoss: 10546.878906
[batchSize = 64, dropOutProb = 0.7, learningRate = 0.01, epoch = 16] loss: 14681.527801, trainLoss: 14682.038086, valLoss: 15201.093750
[batchSize = 64, dropOutProb = 0.7, learningRate = 0.01, epoch = 17] loss: 13779.176620, trainLoss: 13779.042969, valLoss: 10474.831055
[batchSize = 64, dropOutProb = 0.7, learningRate = 0.01, epoch = 18] loss: 13705.555340, trainLoss: 13705.083008, valLoss: 10923.876953
[batchSize = 64, dropOutProb = 0.7, learningRate = 0.01, epoch = 19] loss: 12531.146284, trainLoss: 12530.830078, valLoss: 10137.693359
[batchSize = 64, dropOutProb = 0.7, learningRate = 0.01, epoch = 20] loss: 12592.078398, trainLoss: 12592.112305, valLoss: 10407.869141
[batchSize = 64, dropOutProb = 0.7, learningRate

[batchSize = 64, dropOutProb = 0.7, learningRate = 0.001, epoch = 39] loss: 17485.604155, trainLoss: 17485.230469, valLoss: 12193.019531
[batchSize = 64, dropOutProb = 0.7, learningRate = 0.001, epoch = 40] loss: 16198.301147, trainLoss: 16198.729492, valLoss: 10958.958984
[batchSize = 64, dropOutProb = 0.7, learningRate = 0.001, epoch = 41] loss: 63105.077268, trainLoss: 63106.207031, valLoss: 12079.707031
[batchSize = 64, dropOutProb = 0.7, learningRate = 0.001, epoch = 42] loss: 17080.732904, trainLoss: 17081.007812, valLoss: 12074.814453
[batchSize = 64, dropOutProb = 0.7, learningRate = 0.001, epoch = 43] loss: 15448.304471, trainLoss: 15448.461914, valLoss: 11064.602539
[batchSize = 64, dropOutProb = 0.7, learningRate = 0.001, epoch = 44] loss: 14450.706916, trainLoss: 14450.865234, valLoss: 10032.360352
[batchSize = 64, dropOutProb = 0.7, learningRate = 0.001, epoch = 45] loss: 13808.409884, trainLoss: 13808.478516, valLoss: 9211.549805
[batchSize = 64, dropOutProb = 0.7, learni

[batchSize = 64, dropOutProb = 0.7, learningRate = 0.0001, epoch = 47] loss: 60996.739950, trainLoss: 60996.644531, valLoss: 55410.207031
[batchSize = 64, dropOutProb = 0.7, learningRate = 0.0001, epoch = 48] loss: 59189.527914, trainLoss: 59190.066406, valLoss: 55320.972656
[batchSize = 64, dropOutProb = 0.7, learningRate = 0.0001, epoch = 49] loss: 57388.702631, trainLoss: 57389.488281, valLoss: 56928.191406
[batchSize = 64, dropOutProb = 0.7, learningRate = 0.0001, epoch = 50] loss: 56313.552969, trainLoss: 56312.304688, valLoss: 50367.550781
[batchSize = 64, dropOutProb = 0.7, learningRate = 0.0001, epoch = 51] loss: 52962.182928, trainLoss: 52961.785156, valLoss: 48008.285156
[batchSize = 64, dropOutProb = 0.7, learningRate = 0.0001, epoch = 52] loss: 52410.052738, trainLoss: 52410.125000, valLoss: 48167.914062
[batchSize = 64, dropOutProb = 0.7, learningRate = 0.0001, epoch = 53] loss: 49766.759113, trainLoss: 49767.773438, valLoss: 43974.292969
[batchSize = 64, dropOutProb = 0.7

In [14]:
print (np.min(bestValLoss)**(0.5))

79.48844931320997
